<a href="https://colab.research.google.com/github/GaohuiXu/LLM-NLP/blob/main/Tiny_vs_Giant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>